<a href="https://colab.research.google.com/github/aditya161205/NLP/blob/main/BERT_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###There are two steps in our framework: pre-training and fine-tuning.

During pre-training, the model is trained on unlabeled data over different pre-training tasks. For fine- tuning, the BERT model is first initialized with the pre-trained parameters, and all of the parameters are fine-tuned using labeled data from the downstream tasks

We denote
**number of layers** as **L**

The **hidden size** as **H**

number of **self-attention heads** as **A**.

 We primarily report results on two model sizes: BERTBASE (L=12, H=768, A=12, Total Parameters=110M) and BERTLARGE (L=24, H=1024, A=16, Total Parameters=340M).



---



---
In this project we will use the following values

**L=4**

**H=256**

**A=4**

This makes the total parameters 20M


In [1]:
class Config:
    vocab_size = 30000
    hidden_size = 256
    num_hidden_layers = 4
    num_attention_heads = 4
    intermediate_size = 1024
    max_position_embeddings = 512
    type_vocab_size = 2
    hidden_dropout_prob = 0.1


In [5]:
import numpy
import torch
from datasets import load_dataset

In [13]:
dataset = load_dataset("wikitext", "wikitext-2-v1")


README.md: 0.00B [00:00, ?B/s]

wikitext-2-v1/test-00000-of-00001.parque(…):   0%|          | 0.00/685k [00:00<?, ?B/s]

wikitext-2-v1/train-00000-of-00001.parqu(…):   0%|          | 0.00/6.07M [00:00<?, ?B/s]

wikitext-2-v1/validation-00000-of-00001.(…):   0%|          | 0.00/618k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

### loaded the dataset successfully

In [15]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
train_text=dataset["train"]["text"]

sentences= [sentence for sentence in train_text]

In [20]:
import random

In [25]:
def create_nsp_pairs(sentences, tokenizer,max_len=128):

  nsp_pairs=[]

  for i in range(len(sentences)-1):
    first=sentences[i]

    if random.random()<=0.5:
      second = sentences[i+1]
      label=1
    else:
      second = random.choice(sentences)
      label=2

    encod=tokenizer(first,second,truncation=True,max_length=max_len,padding='max_length',return_tensors='pt')
    nsp_pairs.append({
            "input_ids": encod["input_ids"],
            "token_type_ids": encod["token_type_ids"],
            "attention_mask": encod["attention_mask"],
            "nsp_label": label
        })



In [26]:
nsp=create_nsp_pairs(sentences,tokenizer)